In [22]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# ft = '/mnt/opt/alexw/zjkarina/gpt-j-onlyk_v2'
ft = 'zjkarina/ChatGPTJ_6B'
tokenizer_dialog = AutoTokenizer.from_pretrained(ft)
model_dialog = AutoModelForCausalLM.from_pretrained(ft, torch_dtype=torch.float16, low_cpu_mem_usage=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [23]:
from transformers import AutoModelForPreTraining, AutoTokenizer, BertTokenizer
import torch
tokenizer_sbert = BertTokenizer.from_pretrained("zjkarina/LaBSE-instructDialogs")
print(tokenizer_sbert.vocab_size)
model_sbert = AutoModelForPreTraining.from_pretrained("zjkarina/LaBSE-instructDialogs")
model_sbert.to('cuda')

55083


BertForPreTraining(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(55083, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [24]:
gen_kwargs = {
        "max_new_tokens": 100,
        "top_k": 70,
        "top_p": 0.8,
        "do_sample": True,  
        "no_repeat_ngram_size": 2,
        "bos_token_id": tokenizer_dialog.bos_token_id,
        "eos_token_id": tokenizer_dialog.eos_token_id,
        "pad_token_id": tokenizer_dialog.pad_token_id,
        "temperature": 0.8,
        "use_cache": True,
        "repetition_penalty": 1.2,
        "num_return_sequences": 1
    }

In [25]:
model_dialog.to('cuda')

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): Embedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0-27): 28 x GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (out_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): GPTJMLP(
          (fc_in): Linear(in_features=4096, out_features=16384, bias=True)
          (fc_out): Linear(in_features=16384, out_features=4096, bias=True)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
    )
    (ln_f)

In [26]:
bot_name = input('Enter the name of the bot')
bot_facts = input('Enter the facts about bot')
human_name = input('Enter the your name')
human_facts = input('Enter facts about yourself')

Enter the name of the bot Jim
Enter the facts about bot likes coffee, going for a walk, student
Enter the your name Karina
Enter facts about yourself programmer, likes cats


In [29]:
from sentence_transformers import SentenceTransformer, util
key_senntence = input('how would you like to see the behavior of the chatbot: ')
encoded_key = tokenizer_sbert(key_senntence, padding=True, truncation=True, max_length=24, return_tensors='pt')
with torch.no_grad():
    model_out_key = model_sbert.bert(**encoded_key.to('cuda'))
key_senntence_embeddings = torch.nn.functional.normalize(model_out_key.pooler_output)

how would you like to see the behavior of the chatbot:  nice, interesting speaker, active


In [28]:
persons = f'{bot_name}, who is {bot_facts} talks to {human_name}, who is {human_facts}'
history = []
request = ''
while request != 'stop':
    request = input(f'{human_name}:')
    to_history = ' <|endoftext|> ' + human_name + ': ' + request + ' <|endoftext|> ' + bot_name + ': '
    all_dialog = persons + ' <|endoftext|> ' + ''.join(history) + to_history
    input_ids = tokenizer_dialog.encode(all_dialog, return_tensors='pt').to(model_dialog.device)
    options = []
    for i in range(3): 
        generated = model_dialog.generate(input_ids=input_ids, **gen_kwargs)
        response = tokenizer_dialog.batch_decode(generated[:, len(input_ids[0]):], skip_special_tokens=True)
        options.append(response[0])
    encoded_input = tokenizer_sbert(options, padding=True, truncation=True, max_length=64, return_tensors='pt')
    with torch.no_grad():
        model_output = model_sbert.bert(**encoded_input.to('cuda'))
    embeddings = model_output.pooler_output
    embeddings = torch.nn.functional.normalize(embeddings)
    cos_similarities = util.cos_sim(key_senntence_embeddings, embeddings)[0]
    most_sim = options[cos_similarities.argmax()]
    print(f'{bot_name}:{most_sim}')
    history.append(to_history)
    history.append(response[0])
    if len(history) < 12:
        history = history[:2]


Karina: hi, how are you?


Jim: I'm doing well. How about you, what's your name and do you like any hobbies or interests?


Karina: My name is Karina, I like coding, what about you?


Jim: Nice to meet you! So, can we chat more about our favorite programming languages and websites? What kind of things do they inspire you to create?


Karina: My favorite programming language is python, what do you like?


Jim: Python too! It has such an intuitive syntax that makes it easy to learn. And the community around it is amazing. Have you ever worked with other developers before?


Karina: stop


Jim: Yes, actually. One time I was working on a project with some friends from college, and one day we had a breakthrough moment where everything just clicked into place. It felt so empowering. Do you have any similar experiences?
